In [ ]:
import numpy as np
import healpy as hp
import scipy

from astropy.coordinates import SkyCoord, EarthLocation, get_body
import astropy.units as u
from astropy.time import Time

from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass, plot_sky

import skyproj

import matplotlib.pyplot as plt
#%matplotlib widget

# Candidate Fields

Characteristics of a set of candidate fields

In [ ]:
targets = [
    FixedTarget(SkyCoord(6.022329,-72.081444, unit='deg', frame='icrs'), name='47 Tuc'),
    FixedTarget(SkyCoord(37.9, 7.0, unit='deg', frame='icrs'), name='Rubin_SV_38_7'),
    FixedTarget(SkyCoord(39.9971, -34.4492, unit='deg', frame='icrs'), name='Fornax Dwarf'),
    FixedTarget(SkyCoord(53.125, -28.1, unit='deg', frame='icrs'), name='ECDFS'),
    FixedTarget(SkyCoord(59.1004, -48.73, unit='deg', frame='icrs'), name='EDFS_ComCam'),
    #FixedTarget(SkyCoord(352.360542, -0.844150, unit='deg', frame='icrs'), name='HSC-SSP_Deep2-3'),
    #FixedTarget(SkyCoord(16.0, 4.0, unit='deg', frame='icrs'), name='Ecliptic_16_4'),
    #FixedTarget(SkyCoord(30.0, 10.0, unit='deg', frame='icrs'), name='Ecliptic_30_10'),
    #FixedTarget(SkyCoord(75.0, 8.0, unit='deg', frame='icrs'), name='Ecliptic_75_8'),
    #FixedTarget(SkyCoord(85.0, 10.0, unit='deg', frame='icrs'), name='Ecliptic_85_10'),
    #FixedTarget(SkyCoord(105.0, 20.0, unit='deg', frame='icrs'), name='Ecliptic_105_20'),
    #FixedTarget(SkyCoord(150.1, 2.18194444, unit='deg', frame='icrs'), name='COSMOS'),
    #FixedTarget(SkyCoord(89.26, -16.59, unit='deg', frame='icrs'), name='Rubin_SV_89_-17'), # WD 0554
    #FixedTarget(SkyCoord(98.25, -21.75, unit='deg', frame='icrs'), name='Rubin_SV_98_-22'),
    #FixedTarget(SkyCoord(94.0928496, -21.372688, unit='deg', frame='icrs'), name='Rubin_SV_94_-21'), # NGC 2207
    FixedTarget(SkyCoord(95.0, -25.0, unit='deg', frame='icrs'), name='Rubin_SV_95_-25'),
    #FixedTarget(SkyCoord(100.000000, -34.000000, unit='deg', frame='icrs'), name='Rubin_SV_100_-34'), # Star flat from DES
    #FixedTarget(SkyCoord(110.000000, -22.500000, unit='deg', frame='icrs'), name='Rubin_SV_100_-34'), # Star flat from DES
    #FixedTarget(SkyCoord(112.500000, -50.000000, unit='deg', frame='icrs'), name='Rubin_SV_113_-50'), # Star flat from DES, but out of azimuth range
    #FixedTarget(SkyCoord(103.0, -39.0, unit='deg', frame='icrs'), name='Rubin_SV_103_-30'),
    #FixedTarget(SkyCoord(93.0, -20.0, unit='deg', frame='icrs'), name='Rubin_SV_95_-25'),
    #FixedTarget(SkyCoord(125.0, -15.0, unit='deg', frame='icrs'), name='Rubin_SV_125_-15'),
    #FixedTarget(SkyCoord(95.3621919, -64.9941960, unit='deg', frame='icrs'), name='NGC 2230'), # Galaxy cluster on outskirts of LMC
    #FixedTarget(SkyCoord(15.0392, -33.7089, unit='deg', frame='icrs'), name='Sculptor Dwarf'),
    FixedTarget(SkyCoord(106.23, -10.51, unit='deg', frame='icrs'), name='Seagull'),
]

# Sort by RA
targets.sort(key=lambda _: _.coord.ra.deg, reverse=False)

Ecliptic Latitude

In [ ]:
for target in targets:
    print('%20s%10.3f'%(target.name, target.coord.geocentricmeanecliptic.lat.deg))

Galactic Latitude

In [ ]:
for target in targets:
    print('%20s%10.3f'%(target.name, target.coord.galactic.b.deg))

Example sky viewer:
https://www.legacysurvey.org/viewer?ra=16.0&dec=4.&layer=des-dr1&zoom=6

In [ ]:
ra_target = np.array([target.ra.deg for target in targets])
dec_target = np.array([target.dec.deg for target in targets])

In [ ]:
m_extinction = np.load('/home/b/bechtol/rubin-user/rubin_sim_data/maps/DustMaps/dust_nside_64.npz')

In [ ]:
def plotEcliptic(sp):
    lon = np.linspace(0, 360, 100)
    lat = np.tile(0., 100)
    coord = SkyCoord(lon, lat, unit='deg', frame='geocentricmeanecliptic')
    sp.plot(coord.icrs.ra.deg, coord.icrs.dec.deg, c='black', ls='--', lw=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.Skyproj(ax=ax)
sp.draw_hpxmap(m_extinction['ebvMap'], cmap='Blues', lat_range=(-90., 90.), vmax=0.3, alpha=0.5)
cbar = sp.draw_inset_colorbar(label='E(B-V)', bbox_to_anchor=(-0.05, -0.15, 1, 1), loc='upper right', ticks=(0., 0.1, 0.2), format='%.1f')
plotEcliptic(sp)
sp.scatter(ra_target, dec_target, c='red', s=10, zorder=10)
plt.show()

In [ ]:
print('%20s%10s'%('Target', 'E(B-V)'))
for target in targets:
    ebv = m_extinction['ebvMap'][hp.ang2pix(64, target.ra.deg, target.dec.deg, lonlat=True)]
    print('%20s%10.2f'%(target.name, ebv))

In [ ]:
m_stellar_density = np.load('/home/b/bechtol/rubin-user/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
#plt.figure()
#plt.hist(m_stellar_density['starDensity'][:,20], bins=np.linspace(0, 20000, 101))

In [ ]:
selection = (m_stellar_density['starDensity'][:,20] > 0.)
median_stellar_density = np.median(m_stellar_density['starDensity'][:,20][selection])
#print(median_stellar_density)

In [ ]:
print('%20s%20s'%('Target', 'Stellar Density'))
for target in targets:
    stellar_density = m_stellar_density['starDensity'][:,20][hp.ang2pix(64, target.ra.deg, target.dec.deg, lonlat=True)] / median_stellar_density
    print('%20s%20.2f'%(target.name, stellar_density))

# Visibility

Plot the airmass as a function of time during the night for three different dates over the next month, moving forward in time.

In [ ]:
observer = Observer.at_site("LSST")
observer.location.lat.deg

In [ ]:
time = Time('2024-11-17 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
time = Time('2024-12-01 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
time = Time('2024-12-16 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

# Moon

Check when the Moon gets too close the targets

In [ ]:
time = Time('2024-11-16 03:00:00.000', format='iso')

time_next_month = time + np.arange(0., 30., 1.,) * u.day

lunar_illumination_next_month = observer.moon_illumination(time_next_month)

In [ ]:
x_ticks = Time(np.floor(time_next_month.mjd), format='mjd')
x_tick_labels = np.array([_.iso.split()[0] for _ in x_ticks])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.axhspan(0., 20., alpha=0.2, color='black')

for target in targets:
    lunar_separation_next_month = observer.moon_altaz(time_next_month).separation(observer.altaz(time_next_month, target.coord)).deg
    ax1.plot(time_next_month.mjd, lunar_separation_next_month, label=target.name)

ax1.set_ylabel('Lunar Separation (deg)')
ax1.set_xticks(x_ticks.mjd[::3], x_tick_labels[::3], rotation=45.)
ax1.set_ylim(0., 180.)

ax2.plot(time_next_month.mjd, lunar_illumination_next_month, c='black', ls='--')
ax2.set_ylabel('Lunar Illumination')
ax2.set_ylim(0.,1.)

ax1.legend(loc='upper center')

fig.tight_layout()

# Azimuth at end of Night

After 3am, only targets only pointing towards South or West after 3AM. 
*  observing azimuth shall be between 220 and 330 degrees (for emergency access to dome for closure) 

In [ ]:
time = Time('2024-11-16 08:00:00.000', format='iso')

print('%20s%15s'%('Target', 'Az (deg)'))
for target in targets:
    print('%20s%15.2f'%(target.name, observer.altaz(time, target.coord).az.deg))

In [ ]:
time = Time('2024-12-16 08:00:00.000', format='iso')

print('%20s%15s'%('Target', 'Az (deg)'))
for target in targets:
    print('%20s%15.2f'%(target.name, observer.altaz(time, target.coord).az.deg))

# Sun

In [ ]:
location = EarthLocation.of_site('lsst')

Opposition

In [ ]:
time = Time('2024-11-16 05:00:00.000', format='iso')
sun = get_body('sun', time, location)
sun.ra.deg - 180.

Opposition

In [ ]:
time = Time('2024-12-16 05:00:00.000', format='iso')
sun = get_body('sun', time, location)
sun.ra.deg - 180.

http://people.tamu.edu/~kevinkrisciunas/ra_dec_sun_2024.html

# Visibility Summary

In [ ]:
time = Time('2024-12-03 23:59:00.000', format='iso')

def printTime(time):
    return ':'.join(time.iso.split()[1].split(':')[0:2])


time_midnight = observer.midnight(time, which='nearest')
time_sunset = observer.sun_set_time(time_midnight, which='previous')
time_sunrise = observer.sun_rise_time(time_midnight, which='next')
time_evening_astronomical_twilight = observer.twilight_evening_astronomical(time_midnight, which='previous')
time_morning_astronomical_twilight = observer.twilight_morning_astronomical(time_midnight, which='next')
#time_array = time_sunset + np.arange(0., (time_sunrise - time_sunset).to(u.hr).value, 0.1) * u.hr
time_array = time_evening_astronomical_twilight \
    + np.arange(0., (time_morning_astronomical_twilight - time_evening_astronomical_twilight).to(u.hr).value, 0.1) * u.hr

print('day_obs = %s\n'%(time.iso.split()[0]))

for target in targets:
    time_transit = observer.target_meridian_transit_time(time_midnight, target.coord, which='nearest')
    print(target.name)
    print('  Transit\t%s (UTC)'%(printTime(time_transit)))

    alt_array = observer.altaz(time_array, target.coord).alt.deg
    az_array = observer.altaz(time_array, target.coord).az.deg

    azimuth_exclusion = ((observer.twilight_morning_astronomical(time_midnight, which='next') - time_array).to(u.hour) < (2. * u.hour)) \
        & ~((az_array > 220.) & (az_array < 360.))

    criteria = (alt_array > 45.) \
        & (alt_array < 83.) \
        & (observer.sun_altaz(time_array).alt.value < -18.) \
        & ~azimuth_exclusion \
        & (observer.moon_altaz(time).separation(observer.altaz(time, target.coord)).deg > 30.)

    observable_time_ranges = []
    labels, n_labels = scipy.ndimage.label(criteria)
    for label in range(0, n_labels):
        selection = np.where(labels == (label + 1))[0]

        time_start = time_array[np.min(selection)]
        time_end = time_array[np.max(selection)]
    
        print('  Observable\t%s - %s (UTC)'%(
            #time_start.iso,
            #time_end.iso
            printTime(time_start),
            printTime(time_end),
        ))
    print('')

        #observable_time_ranges.append('%s -- %s'%(
        #    printTime(time_start),
        #    printTime(time_end),
        #))

In [ ]:
airmass_threshold = 1. / np.cos(np.radians(45.))

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time_midnight, brightness_shading=True, altitude_yaxis=True)
ax.axhline(airmass_threshold, c='black', ls='--')
ax.legend(loc='lower right')
plt.title('day_obs = %s\n'%(time.iso.split()[0]))
plt.tight_layout()
plt.savefig('%s_airmass.png'%(time.iso.split()[0]))
plt.show()

In [ ]:
target_moon = FixedTarget(observer.moon_altaz(time_midnight), name='Moon')

plt.clf()
for target in targets:
    plot_sky(target, observer, time_array)
#plot_sky(target_moon, observer, time_array)
plt.title('day_obs = %s\n'%(time.iso.split()[0]))
plt.legend(loc='lower right')
#bbox_to_anchor=(1.25, 0.5))
plt.show()

In [ ]:
if observer.moon_altaz(time_midnight).alt.value > 0.:
    time_moonrise = observer.moon_rise_time(time_midnight, which='previous')
    time_moonset = observer.moon_set_time(time_midnight, which='next')
else:
    time_moonrise = observer.moon_rise_time(time_midnight, which='next')
    time_moonset = observer.moon_set_time(time_midnight, which='previous')

#print('Time (UTC) =', time)

print('%40s%30s'%('Time Evening Civil Twilight (UTC)', observer.twilight_evening_civil(time_midnight, which='previous').iso))
print('%40s%30s'%('Time Evening Nautical Twilight (UTC)', observer.twilight_evening_nautical(time_midnight, which='previous').iso))
print('%40s%30s'%('Time Evening Astronomical Twilight (UTC)', observer.twilight_evening_astronomical(time_midnight, which='previous').iso))
print('%40s%30s'%('Time Midnight (UTC)', time_midnight.iso))
print('%40s%30s'%('Time Morning Astronomical Twilight (UTC)', observer.twilight_morning_astronomical(time_midnight, which='next').iso))
print('%40s%30s'%('Time Morning Nautical Twilight (UTC)', observer.twilight_morning_nautical(time_midnight, which='previous').iso))
print('%40s%30s'%('Time Morning Civil Twilight (UTC)', observer.twilight_morning_civil(time_midnight, which='previous').iso))
print('')
print('%40s%30s'%('Moonrise time (UTC)', time_moonrise.iso))
print('%40s%30s'%('Moonset time (UTC)', time_moonset.iso))
print('%40s%30.2f'%('Moon Illumination', observer.moon_illumination(time)))

# 